In [ ]:
#ipcluster start -n=8

In [1]:
%pylab inline
import numpy as np
import time
import numexpr as ne
import seaborn as sns
from ipywidgets import interact
from sklearn.decomposition import PCA
from numba import jit

Populating the interactive namespace from numpy and matplotlib


In [2]:
ne.set_num_threads(32)

8

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
%gui qt

In [5]:
import sys
sys.path.append('../../..')

In [6]:
from spiketag.mvc import Sorter
from spiketag.base import ProbeFactory, MUA, bload, SPKTAG
from spiketag.utils import conf

# Test 32 liner probe

In [ ]:
nCh = 32
fs = 25000
probe = ProbeFactory.genLinearProbe(fs=fs, n_ch=nCh)
probe

In [ ]:
app = Sorter('/Users/chiy/Documents/HHMI/Github/spiketag/spiketag_test_data/pcie_32_25k.bin',probe = probe, 
              corr_cutoff=0.95, 
              fetlen=6, 
              fet_whiten=False,
              fall_off_size=18, 
              n_jobs=1)
# app = Sorter('/Users/chiy/Documents/HHMI/Github/spiketag/spiketag_test_data/pcie_32_25k.bin', 
#              spktag_filename='/Users/chiy/Documents/HHMI/Github/spiketag/spiketag_test_data/pcie_32_25k.bin.probe')



In [ ]:
#app.model.spktag.tofile('/Users/chiy/Documents/HHMI/Github/spiketag/spiketag_test_data/pcie_32_25k.bin.probe')

In [ ]:
app.run()

# Switch of profiler log, if True, print profiler log

In [ ]:
from spiketag.utils import conf

In [ ]:
conf.ENABLE_PROFILER = True

# Use line profiler to find bottomneck

In [ ]:
app.model.clu[30].selectlist

In [ ]:
app.model.clu[30].index

In [ ]:
# pip install line-profiler first.
%load_ext line_profiler

In [ ]:
%lprun -f app.view.trace_view.locate_and_highlight app.view.trace_view.locate_and_highlight(app.view.spk_view.selected_spk)

# Test 160 tetrode probe

In [7]:
nCh = 64
fs = 25e3
tetrode = ProbeFactory.genTetrodeProbe(fs=fs, n_ch=nCh)
tetrode.reorder_by_chip = False
tetrode

0:[0 1 2 3]
1:[4 5 6 7]
2:[ 8  9 10 11]
3:[12 13 14 15]
4:[16 17 18 19]
5:[20 21 22 23]
6:[24 25 26 27]
7:[28 29 30 31]
8:[32 33 34 35]
9:[36 37 38 39]
10:[40 41 42 43]
11:[44 45 46 47]
12:[48 49 50 51]
13:[52 53 54 55]
14:[56 57 58 59]
15:[60 61 62 63]

In [8]:
app = Sorter('/Users/chiy/Documents/HHMI/Github/spiketag_test_data/ttd_64_25k_20170712_without_noise.bin',probe = tetrode, 
              corr_cutoff=0.95, 
              fet_method='pca',
              fetlen=6, 
              fet_whiten=False,
              fall_off_size=18, 
              spklen=19,
              n_jobs=1)
#app = Sorter('/Users/chiy/Documents/HHMI/Github/spiketag_test_data/ttd_64_25k_20170712_without_noise.bin', spktag_filename='/Users/chiy/Documents/HHMI/Github/spiketag_test_data/ttd_64_25k_20170712_without_noise_spktag.bin')

2017-07-17 15:18:19,646 - spiketag - INFO - load mua data
2017-07-17 15:18:19,661 - spiketag - INFO - #############  load data  ###################
2017-07-17 15:18:19,662 - spiketag - INFO - /Users/chiy/Documents/HHMI/Github/spiketag_test_data/ttd_64_25k_20170712_without_noise.bin loaded, it contains: 
2017-07-17 15:18:19,663 - spiketag - INFO - 7361618 * 64 points (1884574208 bytes) 
2017-07-17 15:18:19,664 - spiketag - INFO - 64 channels with sampling rate of 25000.0000 
2017-07-17 15:18:19,665 - spiketag - INFO - 294.465 secs (4.908 mins) of data
2017-07-17 15:18:19,665 - spiketag - INFO - #############################################
2017-07-17 15:18:21,096 - spiketag - INFO - raw data have 74653 spks
2017-07-17 15:18:21,097 - spiketag - INFO - removing all spks on group which len(spks) less then fetlen
2017-07-17 15:18:21,105 - spiketag - INFO - removed all spks on these groups: {0: 1, 9: 2, 2: 4}
2017-07-17 15:18:21,106 - spiketag - INFO - grouping spike time
2017-07-17 15:18:21

In [9]:
app.run()

2017-07-17 15:18:29,977 - spiketag - WARNING -  group 0 has no spikes! 
2017-07-17 15:18:35,774 - spiketag - WARNING -  group 2 has no spikes! 
2017-07-17 15:18:47,813 - spiketag - INFO - received model modified event, removed spikes[group=7, global_ids=[ 446  500  829 1155 1384 1485]]


In [ ]:
app.model.tofile('/Users/chiy/Documents/HHMI/Github/spiketag_test_data/ttd_64_25k_20170712_without_noise_spktag.bin')

### Detect peak and output pivotal file

In [ ]:
bf = bload(nCh=64, fs=25000)

In [ ]:
bf.load('/Users/chiy/Documents/HHMI/Github/spiketag/spiketag_test_data/tt16bit_160.bin', dtype='int16')

In [ ]:
spks = bf.detect_spks()

In [ ]:
data = bf.asarray()

In [ ]:
from ipywidgets import interact

In [ ]:
@interact(x=(1000,55752704,100), y=(0,63,1))
def plot_wave(x,y):
    plot(data[x-1000:x,y])

In [ ]:
from spiketag.view import wave_view

In [ ]:
data = bf.npmm.reshape(-1,64).astype(np.float32)

In [ ]:
wv = wave_view(data=data, spks=spks)

In [ ]:
wv.show()